# 1) Mount drive, unzip data, clone repo, install packages

## 1.1) Mount Drive and define paths
Run provided colab code to mount Google Drive. Then define dataset paths relative to mount point.

In [ ]:
!rm -rf /content/sample_data
!rm -rf /content/*.jpg
!rm -rf /content/*.png
!rm -rf /content/*.json

In [ ]:
# noinspection PyUnresolvedReferences,PyPackageRequirements
from google.colab import drive
mount_root_abs = '/content/drive'
drive.mount(mount_root_abs)
drive_root = f'{mount_root_abs}/MyDrive'

In [ ]:
import os
# DeepFashion In-shop Clothes Retrieval Benchmark (ICRB)
df_root_drive = f'{drive_root}/Datasets/DeepFashion'
assert os.path.exists(df_root_drive)
df_icrb_root_drive = f'{df_root_drive}/In-shop Clothes Retrieval Benchmark'
assert os.path.exists(df_icrb_root_drive)
df_icrb_img_zip_abs_drive = f'{df_icrb_root_drive}/Img.zip'

# If Img.zip is not present, we need to unzip .../Img/img_iuv.zip directory
# from drive root and then run ICRBScraper.run() from /src/dataset/deep_fashion.
# For this nb, we skip this since it'll take an eternity to complete with
# mounted Google Drive.
assert os.path.exists(df_icrb_img_zip_abs_drive), \
  'Please upload a processed zip (processing img.zip in colab will take' + \
  f' for AGES). \nTried: {df_icrb_img_zip_abs_drive}'

## 1.2) Unzip Img directory in Colab
By unzipping the Img_processed.zip in Colab before running our model we gain significant disk reading speedups.
So, the first step is to unzip images directory, and then save the image directory before proceeding.

In [ ]:
df_icrb_root = df_icrb_root_drive.replace(drive_root, '/content/data')
df_icrb_img_root = f'{df_icrb_root}/Img'
if not os.path.exists(df_icrb_img_root):
  # Clear any previous attempts
  # ATTENTION: This will remove /contents/data/*. So, before running, please make
  # sure no usable files will be deleted.
  !mkdir -p /content/data
  !rm -rf /content/data

  # Create output directory
  !mkdir -p "$df_icrb_root"

  # Transfer Img.zip from Google Drive to Colab
  df_icrb_img_zip_abs = f'{df_icrb_root}/{os.path.basename(df_icrb_img_zip_abs_drive)}'
  if not os.path.exists(df_icrb_img_zip_abs):
    !cp "$df_icrb_img_zip_abs_drive" "$df_icrb_root"
  # Unzip it in Colab
  !unzip -q "$df_icrb_img_zip_abs" -d "$df_icrb_root"
  # Handle newly-created image directory
  assert os.path.exists(df_icrb_img_root), f'df_icrb_img_root: {df_icrb_img_root}'
  assert not os.path.exists(f'{df_icrb_img_root}/Img')
  assert not os.path.exists(f'{df_icrb_img_root}/img')
  !rm -f "$df_icrb_img_zip_abs"
  assert not os.path.exists(df_icrb_img_zip_abs)

## 1.3) Clone github repo
Clone achariso/gans-thesis repo into /content/code
 using git clone.
 For more info see: https://medium.com/@purba0101/how-to-clone-private-github-repo-in-google-colab-using-ssh-77384cfef18f

In [ ]:
repo_root = '/content/code/gans-thesis'
if not os.path.exists(repo_root) and not os.path.exists(f'{repo_root}/requirements.txt'):
    # Check that ssh keys exist
    assert os.path.exists(f'{drive_root}/GitHub Keys')
    id_rsa_abs_drive = f'{drive_root}/GitHub Keys/id_rsa'
    id_rsa_pub_abs_drive = f'{id_rsa_abs_drive}.pub'
    assert os.path.exists(id_rsa_abs_drive)
    assert os.path.exists(id_rsa_pub_abs_drive)
    # On first run: Add ssh key in repo
    if not os.path.exists('/root/.ssh'):
        # Transfer config file
        ssh_config_abs_drive = f'{drive_root}/GitHub Keys/config'
        assert os.path.exists(ssh_config_abs_drive)
        !mkdir -p ~/.ssh
        !cp -f "$ssh_config_abs_drive" ~/.ssh/
        # # Add github.com to known hosts
        !ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts
        # Test: !ssh -T git@github.com

    # Remove any previous attempts
    !rm -rf "$repo_root"
    !mkdir -p "$repo_root"
    # Clone repo
    !git clone git@github.com:achariso/gans-thesis.git "$repo_root"
    src_root = f'{repo_root}/src'
    !rm -rf "$repo_root"/report

## 1.4) Install pip packages
All required files are stored in a requirements.txt files at the repository's root.
Use `pip install -r requirements.txt` from inside the dir to install required packages.

In [ ]:
%cd $repo_root
!pip install -r requirements.txt

In [ ]:
import torch
assert torch.cuda.is_available()

## 1.5) Add code/, */src/ to path
This is necessary in order to be able to run the modules.

In [ ]:
content_root_abs = f'{repo_root}'
src_root_abs = f'{repo_root}/src'
%env PYTHONPATH="/env/python:$content_root_abs:$src_root_abs

# 2) Test code
Test that pulled code is running by running its tests.

In [ ]:
import matplotlib.pyplot as plt

# Fix: Adjust figure size for better plotting in Colab
plt.rcParams["figure.figsize"] = (20, 20)

In [ ]:
%run src/main.py

In [ ]:
test_root = f'{repo_root}/tests'
!python -m unittest discover -s "$test_root" -t "$test_root"


# 3) Train PGPG model on DeepFashion
In this section we give the actual training code for PGPG network. PGPG consists of a 2-stage generator, where each stage is a UNET-like model, and, in our version, a PatchGAN discriminator.

In [ ]:
import torch
import torch.nn as nn
# noinspection PyProtectedMember
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm

## 3.1) Hyper-parameters settings
In this sub-section we define some important training hyper-parameters such as the batch size, and the number of epochs.

In [ ]:
# Training hyperparams
n_epochs = 100
batch_size = 48
lr = 1e-4
adv_criterion = nn.MSELoss()
recon_criterion = nn.L1Loss()

load_shape = 256
load_channels = 3
target_shape = 128
target_channels = 3

train_test_splits = [90, 10]    # for a 90% training - 10% evaluation set split

use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')

# Set save_model to True to have training loop crate model checkpoints at a
# certain interval
save_model = True

# Set the desired LR schedulers for Generator and Discriminator optimizers
# Supported types: 'on_plateau', 'cyclic'
gen_lr_scheduler_type = None
disc_lr_scheduler_type = None

# Step listeners
display_step = 200
checkpoint_step = 500

model_chkpts_root_drive = '/content/drive/MyDrive/Model Checkpoints'

## 3.2) Data
In this sub-section we define the dataset & the dataloader to use. In particular, we use DeepFashion's In-shop Clothes Retrieval Benchmark (ICRB) in cross-pose mode. As so, at each iteration dataloader yields 3 images:


*   `image_1`: the input (condition) image to the generator (from which the styles, clothes, etc. will be "fetched")
*   `image_2`: `image_1` at a different pose and/or scale (from which the output "body structure" and "position" will be "fetched")
*   `dense_pose_2`: the dense pose annotation (as an RGB image) of `image_2`



In [ ]:
from dataset.deep_fashion import ICRBCrossPoseDataloader, ICRBDataset
from utils.pytorch import invert_transforms

# Image transforms:
# If target_shape is different from load one, resize & crop. If target_shape is
# different from load shape, convert to grayscale.
# Update: Now done automatically if you set target_channels, target_shape when
# instantiating the dataloader
#   - len(dataloader) = <number of batches>
#   - len(dataloader.dataset) = <number of total dataset items>
norm_mean = norm_std = 0.5 # params of transforms.Normalize()
gen_transforms = ICRBDataset.get_image_transforms(target_shape, target_channels,
                                                  norm_mean=norm_mean,
                                                  norm_std=norm_std)
# Save inverse transform
gen_transforms_inv = invert_transforms(gen_transforms)

# Define dataloader
skip_pose_norm = True
dl = ICRBCrossPoseDataloader(batch_size=batch_size,
                             image_transforms=gen_transforms,
                             skip_pose_norm=skip_pose_norm,
                             splits=train_test_splits,
                             pin_memory=True)
# Apply basic tests
assert issubclass(dl.__class__, DataLoader)
assert len(dl) == len(dl.dataset) // batch_size + \
                              (1 if len(dl.dataset) % batch_size else 0)
_image_1, _image_2, _dense_pose_2 = next(iter(dl))
assert tuple(_image_1.shape) == (batch_size, target_channels, target_shape, target_shape)
assert tuple(_image_2.shape) == (batch_size, target_channels, target_shape, target_shape)
assert tuple(_dense_pose_2.shape) == (batch_size, target_channels, target_shape, target_shape)

## 3.3) Define PGPG model
In this sub-section we define the PGPG's children network models and in particular:


*   the stage-I generator, `G1`: UNET-like with skip connections, and an FC layer at the bottleneck
*   the stage-II generator, `G2`: UNET-like with skip connections and Dropout at the first half of the "encoder" part of the UNET-like model
*   the whole generator, `G`: composed of the two stages
*   the discriminator, `D`: PatchGAN discriminator



In [ ]:
from modules.discriminators.patch_gan import PatchGANDiscriminator
from modules.generators.pgpg import PGPGGenerator
from utils.train import get_adam_optimizer, weights_init_naive, load_model_chkpt, get_optimizer_lr_scheduler

# Define models
gen = PGPGGenerator(c_in=2*target_channels, c_out=target_channels,
                    w_in=target_shape, h_in=target_shape).to(device)
disc = PatchGANDiscriminator(c_in=2*target_channels,
                             # NOTE: for 5 contracting blocks, output is 4x4
                             n_contracting_blocks=5,
                             use_spectral_norm=True).to(device)
# Define optimizers
# Note: Both generators, G1 & G2, are trained using a joint optimizer
gen_opt = get_adam_optimizer(gen, lr=lr, betas=(0.9, 0.999))
disc_opt = get_adam_optimizer(disc, lr=lr, betas=(0.9, 0.999))

# Initialize weights
gen = gen.apply(weights_init_naive)
disc = disc.apply(weights_init_naive)

# Load checkpoints
if save_model:
    chkpt_images = 0
    state_dict = None
    try:
        chkpt_images, state_dict = load_model_chkpt(model=disc, model_name='pgpg', dict_key='disc', model_opt=disc_opt)
        print('Loaded Discriminator checkpoint!')
    except AssertionError as e:
        print(f'Error while loading Discriminator checkpoint: {str(e)}')

    try:
        if state_dict:
            load_model_chkpt(model=gen, model_name='pgpg', dict_key='gen', model_opt=gen_opt, state_dict=state_dict)
        else:
            chkpt_images, state_dict = load_model_chkpt(model=gen, model_name='pgpg', dict_key='gen', model_opt=gen_opt)
        chkpt_epoch, chkpt_images_in_current_epoch = divmod(chkpt_images, batch_size * len(dl))
        chkpt_step = chkpt_images_in_current_epoch // batch_size
        print('Loaded Generator checkpoint!')
        print(f'\t--> chkpt_images={chkpt_images}, chkpt_epoch={chkpt_epoch}, chkpt_step={chkpt_step}')
    except AssertionError as e:
        print(f'Error while loading Generator checkpoint: {str(e)}')
        chkpt_epoch = chkpt_step = 0

    # Load Dataloader sampler state
    if state_dict and 'dataloader' in state_dict.keys():
        dl.set_state(state_dict['dataloader'])

else:
    chkpt_epoch = chkpt_step = 0

# Learning-rate Schedulers
if gen_lr_scheduler_type:
    if gen_lr_scheduler_type == 'cyclic':
        gen_opt_lr_scheduler = get_optimizer_lr_scheduler(gen_opt, schedule_type=gen_lr_scheduler_type,
                                                          base_lr=lr, max_lr=10 * lr, step_size_up=2 * len(dl),
                                                          mode='exp_range', gamma=0.9,
                                                          cycle_momentum=False,
                                                          last_epoch=chkpt_epoch if 'initial_lr' in gen_opt.param_groups[0].keys() else -1)
    else:
        gen_opt_lr_scheduler = get_optimizer_lr_scheduler(gen_opt, schedule_type=gen_lr_scheduler_type)
else:
    gen_opt_lr_scheduler = None
if disc_lr_scheduler_type:
    if disc_lr_scheduler_type == 'cyclic':
        disc_opt_lr_scheduler = get_optimizer_lr_scheduler(disc_opt, schedule_type=disc_lr_scheduler_type,
                                                           base_lr=lr, max_lr=10 * lr, step_size_up=2 * len(dl),
                                                           mode='exp_range', gamma=0.9,
                                                           cycle_momentum=False,
                                                           last_epoch=chkpt_epoch if 'initial_lr' in disc_opt.param_groups[0].keys() else -1)
    else:
        disc_opt_lr_scheduler = get_optimizer_lr_scheduler(disc_opt, schedule_type=disc_lr_scheduler_type)
else:
    disc_opt_lr_scheduler = None

## 3.4) Evaluation Metrics
The evaluation metrics that we will use to evaluate model performance during an after training are:

* Frechet Inception Distance, FID
* Inception Score, IS
* Structural Similarity Index, (SSIM)
* Precision, Recall and F1, (P, R, F1)

These metrics are saved on each model checkpoint. Below, we initialize the calculators for these metrics.


In [ ]:
from dataset.deep_fashion import ICRBCrossPoseDataset, ICRBDataset

from utils.metrics.fid import FID
from utils.metrics.is_ import IS
from utils.metrics.f1 import F1
from utils.metrics.ssim import SSIM

metrics_n_sample = 512
metrics_batch_sample = 8

metrics_gen_transforms = ICRBDataset.get_image_transforms(target_shape=target_shape, target_channels=target_channels)
metrics_dataset = ICRBCrossPoseDataset(image_transforms=None, pose=True)
metrics_dataset_with_transforms = ICRBCrossPoseDataset(image_transforms=metrics_gen_transforms, pose=True)

fid_calculator = FID(n_samples=metrics_n_sample, batch_size=metrics_batch_sample, device=device)
is_calculator = IS(n_samples=metrics_n_sample, batch_size=metrics_batch_sample, device=device)
f1_calculator = F1(n_samples=metrics_n_sample, batch_size=metrics_batch_sample, device=device)
ssim_calculator = SSIM(n_samples=metrics_n_sample, batch_size=metrics_batch_sample,
                       c_img=target_channels, device=device)

## 3.5) PGPG Training
Finally, let's start training the model! In the following cells we also define methods to print images to be able to preview inference (i.e. generation) quality evolution during training.

In [ ]:
mean_generator_loss = 0
mean_discriminator_loss = 0
cur_step = chkpt_step + chkpt_epoch * len(dl)
first_loop = True
torch.cuda.empty_cache()
for epoch in range(chkpt_epoch, n_epochs):
    for image_1, image_2, pose_2 in tqdm(dl, initial=chkpt_step):
        cur_batch_size = len(image_1)

        image_1 = image_1.to(device)
        image_2 = image_2.to(device)
        pose_2 = pose_2.to(device)


        ##########################################
        ########   Update Discriminator   ########
        ##########################################
        disc_opt.zero_grad()                    # Zero out gradient before backprop
        with torch.no_grad():
            _, g_out = gen(image_1, pose_2)
        disc_loss = disc.get_loss(real=image_2, fake=g_out, condition=image_1,
                                  criterion=adv_criterion)
        disc_loss.backward(retain_graph=True)   # Update discriminator gradients
        disc_opt.step()                         # Update discriminator weights
        # Update LR (if needed)
        if disc_lr_scheduler_type and disc_opt_lr_scheduler:
            disc_opt_lr_scheduler.step(metrics=disc_loss) if disc_lr_scheduler_type == 'on_plateau' \
                else disc_opt_lr_scheduler.step()


        ##########################################
        ########     Update Generator     ########
        ##########################################
        gen_opt.zero_grad()
        g1_loss, g2_loss, g1_out, g_out = gen.get_loss(x=image_1, y=image_2,
                    y_pose=pose_2.clone(), disc=disc, adv_criterion=adv_criterion,
                    recon_criterion=recon_criterion)
        gen_loss = g1_loss + g2_loss
        gen_loss.backward()                     # Update generator gradients

        gen_opt.step()                          # Update generator optimizer
        # Update LR (if needed)
        if gen_lr_scheduler_type and gen_opt_lr_scheduler:
            gen_opt_lr_scheduler.step(metrics=gen_loss) if gen_lr_scheduler_type == 'on_plateau' \
                else gen_opt_lr_scheduler.step()


        ##########################################
        ########      Visualizations      ########
        ##########################################
        # Keep track of the average losses
        mean_discriminator_loss += disc_loss.item() / display_step
        mean_generator_loss += gen_loss.item() / display_step

        # Visualization code
        if cur_step % display_step == 0:
            print(f"\n\t--> Epoch {epoch}: Step {cur_step}: Generator loss: {mean_generator_loss}, Discriminator loss: {mean_discriminator_loss}")

            g1_out_first = gen_transforms_inv(g1_out[0].detach().cpu()).float()
            g1_out_last = gen_transforms_inv(g1_out[-1].detach().cpu()).float()
            g_out_first = gen_transforms_inv(g_out[0].detach().cpu()).float()
            g_out_last = gen_transforms_inv(g_out[-1].detach().cpu()).float()
            g2_out_fist = g_out_first - g1_out_first
            g2_out_last = g_out_last - g1_out_last

            image_1_first = gen_transforms_inv(image_1[0].cpu())
            pose_2_first = pose_2[0].cpu() # No normalization since skip_pose_norm = True
            image_2_first = gen_transforms_inv(image_2[0].cpu())
            image_1_last = gen_transforms_inv(image_1[-1].cpu())
            pose_2_last = pose_2[-1].cpu() # No normalization since skip_pose_norm = True
            image_2_last = gen_transforms_inv(image_2[-1].cpu())

            cat_images = torch.cat((image_1_first, pose_2_first, image_2_first, g1_out_first, g2_out_fist, g_out_first), dim=2)
            cat_images = cat_images.detach().cpu()
            if target_channels == 1:
              cat_images = cat_images.view([cat_images.shape[1], cat_images.shape[2]])
            else:
              cat_images = cat_images.permute(1, 2, 0)
            plt.imshow(cat_images, cmap='gray' if target_channels == 1 else None)
            plt.show()

            cat_images = torch.cat((image_1_last, pose_2_last, image_2_last, g1_out_last, g2_out_last, g_out_last), dim=2)
            cat_images = cat_images.detach().cpu()
            if target_channels == 1:
              cat_images = cat_images.view([cat_images.shape[1], cat_images.shape[2]])
            else:
              cat_images = cat_images.permute(1, 2, 0)
            plt.imshow(cat_images, cmap='gray' if target_channels == 1 else None)
            plt.show()

            mean_generator_loss = 0
            mean_discriminator_loss = 0


        ##########################################
        ########   Metrics & Checkpoint   ########
        ##########################################
        if save_model and not first_loop and cur_step % checkpoint_step == 0:
            # Check if checkpoint already exists
            new_chkpt_path = f'{model_chkpts_root_drive}/pgpg_{str(cur_step).zfill(10)}_{batch_size}.pth'
            if not os.path.exists(new_chkpt_path):
                # Calculate metrics
                gen = gen.eval()
                with torch.no_grad():
                    metrics_fid = fid_calculator(metrics_dataset, gen=gen, gen_transforms=metrics_gen_transforms,
                                                target_index=1, condition_indices=(0, 2))
                    metrics_is = is_calculator(metrics_dataset, gen=gen, gen_transforms=metrics_gen_transforms,
                                                target_index=1, condition_indices=(0, 2))
                    metrics_f1, metrics_p, metrics_r = f1_calculator(metrics_dataset, gen=gen,
                                                                    gen_transforms=metrics_gen_transforms,
                                                                    target_index=1, condition_indices=(0, 2))
                    metrics_ssim = ssim_calculator(metrics_dataset_with_transforms, gen=gen,
                                                  target_index=1, condition_indices=(0, 2),
                                                  skip_asserts=True)
                    metrics_dict = {
                        'fid': metrics_fid.item(),
                        'is': metrics_is.item(),
                        'f1': metrics_f1.item(),
                        'precision': metrics_p.item(),
                        'recall': metrics_r.item(),
                        'ssim': metrics_ssim.item()
                    }
                print(metrics_dict)
                # Save state dicts alongside metrics in a single .pth file
                torch.save({
                    'gen': gen.state_dict(),
                    'gen_opt': gen_opt.state_dict(),
                    'disc': disc.state_dict(),
                    'disc_opt': disc_opt.state_dict(),
                    'metrics': metrics_dict,
                    'dataloader': dl.get_state(),
                }, new_chkpt_path)
                gen = gen.train()

        cur_step += 1
        first_loop = False
    chkpt_step = 0

#####################################
#### Cyclic LR started @ step 29K ###
#####################################


In [ ]:
# Preview metrics
fids = []
iss = []
f1s = []
ssims = []
for _root, _, _files in os.walk(model_chkpts_root_drive):
    for _f in _files:
        if not _f.startswith('pgpg'):
            continue

        metrics = torch.load(f'{_root}/{_f}')['metrics']
        fids.append(metrics['fid'])
        iss.append(metrics['is'])
        f1s.append(metrics['f1'])
        ssims.append(metrics['ssim'])

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10, 5)

plt.plot(fids); plt.title('FID'); plt.show()
plt.plot(iss); plt.title('IS'); plt.show()
plt.plot(ssims); plt.title('SSIM'); plt.show()
plt.plot(f1s); plt.title('F1'); plt.show()

## 3.5) Evaluate Generated Samples
In order to evaluate generated samples and compare model with other GAN architectures trained on the same dataset. For this purpose we will re-calculate the evaluation metrics as stated above, but with a much bigger number of samples. In this way, the metrics will be more trustworthy and comparable with the corresponding metrics in the original paper.


In [ ]:
# Calculate final metrics
metrics_n_sample = 10000
metrics_batch_sample = 64

fid_calculator = FID(n_samples=metrics_n_sample, batch_size=metrics_batch_sample, device=device)
is_calculator = IS(n_samples=metrics_n_sample, batch_size=metrics_batch_sample, device=device)
f1_calculator = F1(n_samples=metrics_n_sample, batch_size=metrics_batch_sample, device=device)
ssim_calculator = SSIM(n_samples=metrics_n_sample, batch_size=metrics_batch_sample, device=device, c_img=target_channels)

gen = gen.eval()
torch.cuda.empty_cache()
with torch.no_grad():
    metrics_fid = fid_calculator(metrics_dataset, gen=gen, gen_transforms=metrics_gen_transforms,
                                 target_index=1, condition_indices=(0, 2))
    metrics_is = is_calculator(metrics_dataset, gen=gen, gen_transforms=metrics_gen_transforms,
                                 target_index=1, condition_indices=(0, 2))
    metrics_f1, metrics_p, metrics_r = f1_calculator(metrics_dataset, gen=gen,
                                                     gen_transforms=metrics_gen_transforms,
                                                     target_index=1, condition_indices=(0, 2),
                                                     row_batch_size=16, col_batch_size=16)
    metrics_ssim = ssim_calculator(metrics_dataset_with_transforms, gen=gen,
                                   target_index=1, condition_indices=(0, 2),
                                   skip_asserts=True)

print(str({
    'fid': metrics_fid.item(),
    'is': metrics_is.item(),
    'f1': metrics_f1.item(), 'precision': metrics_p.item(), 'recall': metrics_r.item(),
    'ssim': metrics_ssim.item()
}))


######################################
######################################
##                                  ##
## At epoch=3, the metrics were:    ##
## {                                ##
##   "fid": 283.1460876464844       ##
##   "is": 3.959059476852417        ##
##   "f1": 0.2747354805469513       ##
##   "precision": 0.18958996236324  ##
##   "recall": 0.49870622158050537  ##
##   "ssim": 0.6973406672477722     ##
## }                                ##
##                                  ##
######################################
######################################

